## Project Details

The work is an attempt to build a CNN model to identify if the given X-ray image of chest in infected with Pneumonai or not. There are two categories of datasets that are covid patient with pneumonia and covid patient without pneumonia. The main objective is to improve Recall score because we would not want to miss the patient infected with Pneumonia and thus left untreated.
Please follow the link to access the dataset : https://www.kaggle.com/rashikrahmanpritom/covid-wwo-pneumonia-chest-xray

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import gc

from keras.preprocessing import image
from keras.models import Sequential
import joblib
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

In [2]:
# # Tracing path of the image files
# covid_with_pn=Path("E:\DS Projects\Deep_Learning\Data\train\covid_with_PNEUMONIA")
# covid_without_pn=Path("E:\DS Projects\Deep_Learning\Data\train\covid_without_PNEUMONIA")

In [3]:
# Tracing path of the image files
covid_with_pn=Path("Data/train/covid_with_PNEUMONIA")
covid_without_pn=Path("Data/train/covid_without_PNEUMONIA")

In [20]:
# Labelling training data images with Pneumonia as "1" and that of without Pneumonia as "0"
w = 256; h=256
images, labels=[],[]
for i in covid_with_pn.glob("*.*"):
    data=image.load_img(i,target_size=(w, h),interpolation="bilinear")
    img=image.img_to_array(data)
    images.append(img)
    labels.append(1)

for i in covid_without_pn.glob("*.*"):
    data=image.load_img(i,target_size=(w,h),interpolation="bilinear")
    img=image.img_to_array(data)
    images.append(img)
    labels.append(0)

In [21]:
test_covid_with_pn=Path("Data/test/covid_with_PNEUMONIA")
test_covid_without_pn=Path("Data/test/covid_without_PNEUMONIA")

In [22]:
# Labelling testing data images with Pneumonia as "1" and that of without Pneumonia as "0"
test_images, test_labels=[],[]

for i in test_covid_with_pn.glob("*.*"):
    data=image.load_img(i,target_size=(w,h),interpolation="bilinear")
    img=image.img_to_array(data)
    test_images.append(img)
    test_labels.append(1)
    
for i in test_covid_without_pn.glob("*.*"):
    data=image.load_img(i,target_size=(w,h),interpolation="bilinear")
    img=image.img_to_array(data)
    test_images.append(img)
    test_labels.append(0)

In [23]:
x_train=np.array(images)/255
y_train=np.array(labels)
x_test=np.array(test_images)/255
y_test=np.array(test_labels)

In [24]:
gc.collect()

1568

# CNN Model


 Building the Convolution Neural Network architecture

In [26]:
# Function to get the f1 score of the model
# def get_f1(y_true, y_pred): #taken from old keras source code
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     recall = true_positives / (possible_positives + K.epsilon())
#     f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
#     return f1_val

In [27]:
from tensorflow import keras
import keras.backend as K

In [28]:
model=Sequential()
model.add(Conv2D(32,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(w,h,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2,2)))
# model.add(Conv2D(128,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(MaxPooling2D((2,2)))
model.add(Flatten())
# model.add(Dropout(0.4))
model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(1,activation="sigmoid"))

In [29]:
optimizers=keras.optimizers.Adamax(learning_rate=0.01)
model.compile(optimizer=optimizers, loss="binary_crossentropy", metrics=['Precision', 'Recall', 'accuracy'] )

In [30]:
model.fit(x_train,y_train, batch_size = 10, epochs = 8, validation_data=(x_test,y_test))

Epoch 1/8
45/45 [==============================] - 25s 551ms/step - loss: 54.6677 - precision: 0.7970 - recall: 0.8618 - accuracy: 0.8044 - val_loss: 0.2634 - val_precision: 0.8696 - val_recall: 0.9302 - val_accuracy: 0.8889
Epoch 2/8
45/45 [==============================] - 25s 547ms/step - loss: 0.3480 - precision: 0.9020 - recall: 0.8984 - accuracy: 0.8911 - val_loss: 0.2744 - val_precision: 0.9487 - val_recall: 0.8605 - val_accuracy: 0.9012
Epoch 3/8
45/45 [==============================] - 25s 547ms/step - loss: 0.2900 - precision: 0.9280 - recall: 0.8902 - accuracy: 0.9022 - val_loss: 1.0488 - val_precision: 1.0000 - val_recall: 0.3488 - val_accuracy: 0.6543
Epoch 4/8
45/45 [==============================] - 25s 546ms/step - loss: 0.1944 - precision: 0.9328 - recall: 0.9593 - accuracy: 0.9400 - val_loss: 0.1180 - val_precision: 0.9149 - val_recall: 1.0000 - val_accuracy: 0.9506
Epoch 5/8
45/45 [==============================] - 25s 552ms/step - loss: 0.1704 - precision: 0.9323 - 

In [31]:
gc.collect()

1579

In [46]:
from sklearn.metrics import f1_score

In [48]:
#Calculated F1 score on validation data.
f1_score(y_test,pred_lables)

0.9662921348314606

## Evaluating the model performance on Validation data.

In [41]:
pred_lables=model.predict(x_test)
pred_lables = pred_lables>0.5
gc.collect()
confusion_matrix(y_test, pred_lables.T[0])

array([[35,  3],
       [ 0, 43]], dtype=int64)

## Evaluating the model performance on Training data.

In [42]:
pred_lables_train=model.predict(x_train)
pred_lables_train = pred_lables_train>0.5
gc.collect()
confusion_matrix(y_train, pred_lables_train.T[0])

array([[201,   3],
       [  2, 244]], dtype=int64)